In [4]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16,ResNet50,DenseNet201,ResNet152V2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import shutil
import cv2
import os
import time
import tensorflow as tf
random_value=42 #setting randomness at fixed for every run
from sklearn import metrics
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method
from cleverhans.tf2.attacks.fgnm import fgnm
from cleverhans.tf2.attacks.pgdnm import pgdnm
from cleverhans.tf2.attacks.bimnm import bimnm
from cleverhans.tf2.attacks.basic_iterative_method import basic_iterative_method

from math import log
c = time.time()
data = np.load("data1.npy")
labels = np.load("label_precess.npy")
d = time.time()
print(d-c)
(trainX1, testX1, trainY1, testY1) = train_test_split(data, labels, test_size=0.2,stratify=labels, random_state=random_value)
# initialize the training data augmentation object
print("Size of train image : {} and train label : {}".format(len(trainX1),len(trainY1)))
print("Size of test image : {} and test label : {}".format(len(testX1),len(testY1)))
trainX = tf.convert_to_tensor(trainX1)
testX = tf.convert_to_tensor(testX1)
trainY = tf.convert_to_tensor(trainY1)
testY = tf.convert_to_tensor(testY1)
testX2 = tf.cast(testX,dtype=tf.float32)

2.730069398880005
Size of train image : 3600 and train label : 3600
Size of test image : 900 and test label : 900


2022-05-20 01:34:28.565129: W tensorflow/core/common_runtime/bfc_allocator.cc:456] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.01GiB (rounded to 1083801600)requested by op Cast
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-05-20 01:34:28.565209: I tensorflow/core/common_runtime/bfc_allocator.cc:991] BFCAllocator dump for GPU_0_bfc
2022-05-20 01:34:28.565235: I tensorflow/core/common_runtime/bfc_allocator.cc:998] Bin (256): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2022-05-20 01:34:28.565243: I tensorflow/core/common_runtime/bfc_allocator.cc:998] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2022-05-20 01:34:28.565267: I tensorflow/core/common_runtime/bfc_allo

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run Cast: Dst tensor is not initialized. [Op:Cast]

In [2]:
model_vgg=tf.keras.models.load_model('./model/vgg160514.h5')
predict_vgg16=np.argmax(model_vgg.predict(testX),1)
print("VGG16 acc:{}".format(metrics.accuracy_score(np.argmax(testY,1),predict_vgg16)))
model_iv3=tf.keras.models.load_model('./model/iv30514.h5')
predict_iv3=np.argmax(model_iv3.predict(testX),1)
print("iv3 acc:{}".format(metrics.accuracy_score(np.argmax(testY,1),predict_iv3)))
model_den=tf.keras.models.load_model('./den.h5')
predict_den=np.argmax(model_den.predict(testX),1)
print("den acc:{}".format(metrics.accuracy_score(np.argmax(testY,1),predict_den)))

2022-05-19 22:17:29.899213: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-05-19 22:17:29.901814: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2794690000 Hz
2022-05-19 22:17:30.213808: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2022-05-19 22:17:30.712897: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8101
2022-05-19 22:17:31.368551: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-05-19 22:17:31.810151: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2022-05-19 22:17:33.612681: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


VGG16 acc:0.9288888888888889
iv3 acc:0.9022222222222223
den acc:0.9211111111111111


In [3]:
tf.keras.models.load_model('./freeze_dens.h5')

ValueError: No model found in config file.

In [3]:
def acc(realdata,pred):
    real_label = np.argmax(realdata,axis=1)
    #pred = np.argmax(pred,axis=1)
    return metrics.accuracy_score(real_label,pred)

###这里还要记录攻击过后图像本身的值
#  我们不只需要攻击过后在白盒测试上的结果，我们还需要黑盒测试的结果。 
def pgd(model_white, model_black1, model_black2 ,x,eps,eps_iter,nb_iter,targeted=False):
    pred_total_vgg = []
    for i in x:
        temp_image = i.numpy().reshape(1,224,224,3)
        temp_image_withatk = projected_gradient_descent(model_white, temp_image,eps,eps_iter,nb_iter,np.inf,targeted=False)
        pred_vgg = model_white.predict(temp_image_withatk)
        pred_vgg =  np.argmax(pred_vgg,axis=1)
        pred_total_vgg.append(pred_vgg)
        
        pred_iv3 = model_black1.predict(temp_image_withatk)
        pred_iv3 =  np.argmax(pred_iv3,axis=1)
        pred_total_iv3.append(pred_iv3)
        
        pred_den = model_black2.predict(temp_image_withatk)
        pred_den =  np.argmax(pred_den,axis=1)
        pred_total_den.append(pred_den)
        
    return pred_total_vgg, pred_total_iv3, pred_total_den

def pgd_nm(model_white, model_black1, model_black2 ,x,eps,eps_iter,nb_iter,targeted=False):
    pred_total_vgg = []
    for i in x:
        temp_image = i.numpy().reshape(1,224,224,3)
        temp_image_withatk = pgdnm(model_white, temp_image,eps,eps_iter,nb_iter,np.inf,targeted=False)
        pred_vgg = model_white.predict(temp_image_withatk)
        pred_vgg =  np.argmax(pred_vgg,axis=1)
        pred_total_vgg.append(pred_vgg)
        
        pred_iv3 = model_black1.predict(temp_image_withatk)
        pred_iv3 =  np.argmax(pred_iv3,axis=1)
        pred_total_iv3.append(pred_iv3)
        
        pred_den = model_black2.predict(temp_image_withatk)
        pred_den =  np.argmax(pred_den,axis=1)
        pred_total_den.append(pred_den)
        
    return pred_total_vgg, pred_total_iv3, pred_total_den

def fgm(model,x,eps,targeted=False):
    pred_total = []
    for i in x:
        #a = time.time()
        temp_image = i.numpy().reshape(1,224,224,3)
        temp_image_withatk = fast_gradient_method(model, temp_image,eps,np.inf,targeted=False)
        pred = model.predict(temp_image_withatk)
        pred =  np.argmax(pred,axis=1)
        pred_total.append(pred)
        #b = time.time()
        #print("check one time:", b-a)
    return pred_total
def fgm_nm(model,x,eps,targeted=False):
    pred_total = []
    for i in x:
        temp_image = i.numpy().reshape(1,224,224,3)
        temp_image_withatk = fgnm(model, temp_image,eps,np.inf,targeted=False)
        pred = model.predict(temp_image_withatk)
        pred =  np.argmax(pred,axis=1)
        pred_total.append(pred)
    return pred_total
def bim(model,x,eps,eps_iter,nb_iter,targeted=False):
    pred_total = []
    for i in x:
        temp_image = i.numpy().reshape(1,224,224,3)
        temp_image_withatk = basic_iterative_method(model,temp_image,eps,eps_iter,nb_iter,np.inf,targeted=False)
        pred = model.predict(temp_image_withatk)
        pred =  np.argmax(pred,axis=1)
        pred_total.append(pred)
    return pred_total
def bim_nm(model,x,eps,eps_iter,nb_iter,targeted=False):
    pred_total = []
    for i in x:
        temp_image = i.numpy().reshape(1,224,224,3)
        temp_image_withatk = bimnm(model,temp_image,eps,eps_iter,nb_iter,np.inf,targeted=False)
        pred = model.predict(temp_image_withatk)
        pred =  np.argmax(pred,axis=1)
        pred_total.append(pred)
    return pred_total

In [4]:
def pp(model,x):
    real_pred = model.predict(testX)
    real_pred = np.argmax(real_pred,axis=1)
    real_label = np.argmax(testY,axis=1)
    
    pred_with_atk = np.array(x).reshape(-1)
    
    
    
    adv_alg_pred = model.predict(x)
    pred_with_atk = np.argmax(adv_alg_pred,axis=1)
    
    from sklearn import metrics
    print("原来的预测精度为{}".format(metrics.accuracy_score(real_pred,real_label)))
    print("对抗后的预测精度为{}".format(metrics.accuracy_score(pred_with_atk,real_label)))
    print()
    return metrics.accuracy_score(pred_with_atk,real_label),metrics.accuracy_score(real_pred,real_label)

In [6]:
### FGSM FGNM
### model VGG16      model2 Densenet        model4    Incv3
testX2 = tf.cast(testX,dtype=tf.float32)
# ,clip_min=0,clip_max=1
eps = [0, 0.001, 0.002, 0.004, 0.005,0.006, 0.008, 0.01, 0.015, 0.02, 0.03,0.05,0.07,0.09,0.1]
for i in eps:
    pred_fgsm = fgm(model_vgg,testX,i)
    pred_fgsmnm = fgm_nm(model_vgg,testX,i)
    print("epsilon = {}".format(i))
    print("-----------VGG fgsm------------")
    pp(model_vgg,pred_fgsm)
    print("------------VGG fgnm------------")
    pp(model_vgg,pred_fgsmnm)
    print("-----------IV3 fgsm------------")
    pp(model_iv3,pred_fgsm)
    print("------------IV3 fgnm------------")
    pp(model_iv3,pred_fgsmnm)
    print("-----------DEN fgsm------------")
    pp(model_den,pred_fgsm)
    print("------------DEN fgnm------------")
    pp(model_den,pred_fgsmnm)
    print("**********************************************")


epsilon = 0
-----------VGG fgsm------------
原来的预测精度为0.9288888888888889
对抗后的预测精度为0.9288888888888889

------------VGG fgnm------------
原来的预测精度为0.9288888888888889
对抗后的预测精度为0.9288888888888889

-----------IV3 fgsm------------
原来的预测精度为0.9022222222222223
对抗后的预测精度为0.9288888888888889

------------IV3 fgnm------------
原来的预测精度为0.9022222222222223
对抗后的预测精度为0.9288888888888889

-----------DEN fgsm------------
原来的预测精度为0.9544444444444444
对抗后的预测精度为0.9288888888888889

------------DEN fgnm------------
原来的预测精度为0.9544444444444444
对抗后的预测精度为0.9288888888888889

**********************************************
epsilon = 0.001
-----------VGG fgsm------------
原来的预测精度为0.9288888888888889
对抗后的预测精度为0.5911111111111111

------------VGG fgnm------------
原来的预测精度为0.9288888888888889
对抗后的预测精度为0.29888888888888887

-----------IV3 fgsm------------
原来的预测精度为0.9022222222222223
对抗后的预测精度为0.5911111111111111

------------IV3 fgnm------------
原来的预测精度为0.9022222222222223
对抗后的预测精度为0.29888888888888887

-----------DEN fgsm------------
原来的预测精

In [ ]:
testX2 = tf.cast(testX,dtype=tf.float32)
# adv_pgd = projected_gradient_descent(model4, testX2,1, 0.1,10, 2, targeted=False)
# adv_pgdnm  = pgdnm(model4, testX2,1, 0.1,10, 2, targeted=False)
# adv_fgm = fast_gradient_method(model4, testX, 1, 2, targeted=False)
# adv_fgnm = fgnm(model4,testX,1, 2, targeted=False)
# adv_bim = basic_iterative_method(model4,testX2,1,0.1,10,2,targeted=False)
# adv_bimnm = bimnm(model4,testX2,1,0.1,10,2,targeted=False)
# 

# 生成对抗样本  model是攻击的model
def create_adv(model,X,eps,eps_iter,nb_iter):    
    adv_pgd = projected_gradient_descent(model, X,eps,eps_iter,nb_iter,np.inf,targeted=False)
    adv_pgdnm  = pgdnm(model,X,eps,eps_iter,nb_iter,np.inf,targeted=False)
    # adv_fgm = fast_gradient_method(model, X,eps,np.inf,targeted=False)
    # adv_fgnm = fgnm(model,X,eps, np.inf,targeted=False)
    adv_bim = basic_iterative_method(model,X,eps,eps_iter,nb_iter,np.inf,targeted=False)
    adv_bimnm = bimnm(model,X,eps,eps_iter,nb_iter,np.inf,targeted=False)
    return adv_pgd,adv_pgdnm,adv_bim,adv_bimnm

# 真实情况与对抗后的样本对比  model这里是测试的model
def pp(model,x):
    real_pred = model.predict(testX)
    real_pred = np.argmax(real_pred,axis=1)
    real_label = np.argmax(testY,axis=1)
    adv_alg_pred = model.predict(x)
    pred_with_atk = np.argmax(adv_alg_pred,axis=1)
    print()
    from sklearn import metrics
    print("对抗后的预测精度为{}".format(metrics.accuracy_score(pred_with_atk,real_label)))
    print("原来的预测精度为{}".format(metrics.accuracy_score(real_pred,real_label)))
    return metrics.accuracy_score(pred_with_atk,real_label),metrics.accuracy_score(real_pred,real_label)
    
# 给一个测试情况下的model
def printf(model):
    # print("-----------fgm------------")
    # pp(model,adv_fgm)
    # print("------------fgnm------------")
    # pp(model,adv_fgnm)
    print("-----------pgd------------")
    pp(model,adv_pgd)
    print("-----------pgdnm------------")
    pp(model,adv_pgdnm)
    print("------------bim--------------")
    pp(model,adv_bim)
    print("------------bimnm--------")
    pp(model,adv_bimnm)

In [ ]:
iter_eps = [0.00001, 0.00003,0.00006, 0.0001, 0.0003, 0.0006, 0.001, 0.003]
for i in iter_eps:
    print("iter_eps = {}".format(i))
    print("VGG16--VGG16")
    adv_pgd,adv_pgdnm,adv_bim,adv_bimnm = create_adv(model1,testX2,1,i,20)
    printf(model1)
    print("VGG16--Inv3")
    #adv_fgm,adv_fgnm,adv_pgd,adv_pgdnm,adv_bim,adv_bimnm = create_adv(model1,testX2,1,i,20)
    printf(model4)
    print("VGG16--Dens")
    #adv_fgm,adv_fgnm,adv_pgd,adv_pgdnm,adv_bim,adv_bimnm = create_adv(model1,testX2,1,i,20)
    printf(model2)
    print("****************************************************************************************")

In [3]:
pred_pgd = pgd(model_vgg,testX,0.1,0.01,10)

In [4]:
pred_bim = bim(model_vgg,testX2,0.1,0.01,10)

In [15]:
print(np.array(pred_pgd).reshape(-1))

[1 0 1 1 1 1 0 1 1 1 2 1 1 1 0 1 0 1 1 1 0 0 1 2 1 1 0 0 0 0 1 1 0 1 0 0 0
 1 2 2 1 0 0 1 1 0 1 1 1 0 0 0 0 1 0 2 1 1 0 1 1 0 1 0 0 0 1 0 0 1 1 0 0 0
 1 0 1 0 0 0 0 1 1 1 0 1 1 0 0 0 0 1 2 1 1 2 0 0 1 1 0 1 0 0 0 1 0 1 0 1 2
 1 1 1 1 2 0 1 2 2 0 1 1 2 1 1 0 1 0 0 1 1 0 2 2 0 2 1 0 1 0 1 1 0 1 1 1 0
 0 0 1 0 0 0 0 0 0 2 1 0 0 0 1 0 1 0 0 0 0 1 0 1 0 1 1 2 1 1 0 0 0 0 1 0 0
 1 0 0 0 1 1 0 1 1 2 0 1 1 1 1 0 0 1 0 1 0 1 0 0 1 0 0 2 1 0 0 1 1 0 0 0 0
 1 2 1 0 0 1 0 0 2 0 1 0 1 0 1 0 2 1 0 1 0 1 0 1 1 0 0 0 1 0 1 1 0 1 1 0 1
 0 1 0 1 1 1 1 0 1 1 1 0 1 2 1 0 1 0 0 0 0 1 1 1 0 1 0 0 0 1 0 1 0 0 1 1 1
 0 0 0 0 0 0 1 0 0 1 0 0 0 1 1 0 1 1 1 0 0 1 0 0 0 0 1 0 0 0 1 1 2 0 1 0 0
 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1 0 1 1 0 0 1 1 0 1 1
 0 1 0 1 0 1 1 1 1 0 0 0 2 1 1 0 2 2 0 0 0 1 0 1 0 0 1 0 1 1 2 1 1 0 0 2 0
 0 1 1 1 1 2 0 1 1 1 1 1 0 1 1 1 0 0 0 1 1 0 1 0 0 1 0 1 0 1 1 1 1 0 1 1 1
 0 1 1 1 2 1 0 1 0 1 0 1 1 1 0 2 1 2 0 1 1 1 0 0 1 0 0 2 1 0 0 1 1 1 1 1 1
 1 0 1 1 1 0 1 0 1 0 1 0 

In [11]:
print(np.argmax(testY,1))

[0 2 2 0 0 2 1 0 0 2 0 0 0 0 2 0 1 0 0 0 2 2 0 0 2 2 1 1 1 2 0 0 2 0 1 2 1
 2 1 1 2 1 2 2 0 1 2 0 0 2 1 1 1 0 2 1 0 0 2 0 0 1 0 1 1 2 0 1 2 0 0 1 1 1
 0 1 0 1 2 1 0 2 2 0 2 0 0 1 1 1 2 0 1 0 0 0 1 1 1 2 1 2 1 1 1 0 1 0 1 0 1
 0 2 2 0 1 2 2 1 1 1 2 0 1 2 1 1 2 2 2 0 0 1 0 1 2 1 0 2 0 2 0 0 1 0 0 2 1
 2 1 2 2 1 2 2 2 1 0 1 2 1 2 0 2 2 1 0 2 1 2 0 2 2 0 2 1 1 0 1 1 1 2 0 2 2
 2 2 2 2 0 2 1 0 2 0 1 2 0 0 2 1 2 2 1 2 1 2 1 1 0 1 2 1 0 2 1 0 2 0 1 1 1
 2 1 0 1 1 0 1 2 1 2 0 1 0 1 0 2 1 0 1 0 1 0 2 0 0 1 2 1 2 1 0 2 2 0 0 2 2
 1 2 1 2 0 1 0 1 1 0 2 1 0 1 2 0 2 2 1 1 1 2 0 2 1 0 2 2 1 1 2 0 2 2 0 2 0
 0 1 2 1 2 1 2 1 1 0 1 2 1 0 2 1 2 0 0 2 2 0 2 1 1 1 0 1 2 1 0 0 1 0 0 2 2
 1 2 0 2 2 1 2 1 2 0 2 2 2 1 2 2 2 1 1 0 0 2 2 1 0 0 0 1 0 0 1 1 0 0 1 0 0
 1 0 2 2 2 2 2 2 0 2 1 1 1 0 0 2 1 1 1 0 0 2 2 2 2 1 0 1 1 0 1 0 2 1 1 1 1
 2 2 0 2 0 1 1 1 2 1 0 2 1 0 2 2 1 2 1 2 2 1 2 2 1 0 1 0 1 0 0 2 0 2 0 2 2
 1 0 0 0 1 2 0 0 2 0 0 0 2 2 1 1 2 0 2 0 0 0 2 1 0 1 2 1 0 1 2 0 0 0 2 2 2
 0 1 0 0 2 1 0 1 0 1 2 2 

In [17]:
print(metrics.accuracy_score(np.argmax(testY,1), np.array(pred_pgd).reshape(-1)))

0.043333333333333335


In [18]:
from cleverhans.tf2.attacks.fgnm import fgnm
from cleverhans.tf2.attacks.pgdnm import pgdnm
from cleverhans.tf2.attacks.bimnm import bimnm

In [24]:
print(len(pred_pgdnm))

900


In [22]:
print(metrics.accuracy_score(np.argmax(testY,1), np.array(pred_pgdnm).reshape(-1)))

0.2822222222222222


In [43]:
eps=[0.0005,0.0008,0.001, 0.002, 0.003,0.004, 0.005, 0.008, 0.01]
for i in eps:
    print(i)
    a = time.time()
    pred_fgsm = fgm(model_vgg,testX,i)
    pred_fgsmnm = fgm_nm(model_vgg,testX,i)
    print(metrics.accuracy_score(np.argmax(testY,1), np.array(pred_fgsm).reshape(-1)))
    print(metrics.accuracy_score(np.argmax(testY,1), np.array(pred_fgsmnm).reshape(-1)))
    b = time.time()
    print("time one eps:", b-a)
    print("\n")

0.0005
0.8322222222222222
0.5788888888888889
time one eps: 135.76903796195984


0.0008
0.6933333333333334
0.37444444444444447
time one eps: 137.56198644638062


0.001
0.5911111111111111
0.29888888888888887
time one eps: 135.46983361244202


0.002
0.2911111111111111
0.16111111111111112
time one eps: 135.18053698539734


0.003
0.18555555555555556
0.12555555555555556
time one eps: 136.46077489852905


0.004
0.1288888888888889
0.11222222222222222
time one eps: 136.80946493148804


0.005
0.10888888888888888
0.10666666666666667
time one eps: 135.41818690299988


0.008
0.08333333333333333
0.09888888888888889
time one eps: 135.6816701889038


0.01
0.07666666666666666
0.09666666666666666
time one eps: 137.81809282302856




In [ ]:
eps=[0.01, 0.015, 0.02, 0.03, 0.05, 0.08]
for i in eps:
    print(i)
    a = time.time()
    pred_fgsm = fgm(model_vgg,testX,i)
    pred_fgsmnm = fgm_nm(model_vgg,testX,i)
    print(metrics.accuracy_score(np.argmax(testY,1), np.array(pred_fgsm).reshape(-1)))
    print(metrics.accuracy_score(np.argmax(testY,1), np.array(pred_fgsmnm).reshape(-1)))
    b = time.time()
    print("time one eps:", b-a)
    print("\n")

In [4]:
iter_eps = [0.0003,0.0004,0.0005, 0.0006, 0.0008, 0.001, 0.003]
for i in iter_eps:
    print(i)
    a = time.time()
    pred_pgd = pgd(model_vgg,testX,1,i,20)
    pred_pgdnm = pgd_nm(model_vgg,testX,1,i,20)
    print(metrics.accuracy_score(np.argmax(testY,1), np.array(pred_pgd).reshape(-1)))
    print(metrics.accuracy_score(np.argmax(testY,1), np.array(pred_pgdnm).reshape(-1)))
    b = time.time()
    print("time one eps:", b-a)
    print("\n")

0.0003
0.07
0.09555555555555556
time one eps: 425.971727848053


0.0004
0.07
0.09555555555555556
time one eps: 426.4157633781433


0.0005
0.06555555555555556
0.09555555555555556
time one eps: 430.9538764953613


0.0006
0.06666666666666667
0.09555555555555556
time one eps: 432.1830930709839


0.0008
0.06777777777777778
0.09555555555555556
time one eps: 426.24455738067627


0.001


KeyboardInterrupt: 

In [ ]:
def pgd(model_white, model_black1, model_black2 ,x,eps,eps_iter,nb_iter,targeted=False):
    pred_total_vgg = []
    for i in x:
        temp_image = i.numpy().reshape(1,224,224,3)
        temp_image_withatk = projected_gradient_descent(model_white, temp_image,eps,eps_iter,nb_iter,np.inf,targeted=False)
        pred_vgg = model_white.predict(temp_image_withatk)
        pred_vgg =  np.argmax(pred_vgg,axis=1)
        pred_total_vgg.append(pred_vgg)
        
        pred_iv3 = model_black1.predict(temp_image_withatk)
        pred_iv3 =  np.argmax(pred_iv3,axis=1)
        pred_total_iv3.append(pred_iv3)
        
        pred_den = model_black2.predict(temp_image_withatk)
        pred_den =  np.argmax(pred_den,axis=1)
        pred_total_den.append(pred_den)
        
    return pred_total_vgg, pred_total_iv3, pred_total_den

###这里还要记录攻击过后图像本身的值
#  我们不只需要攻击过后在白盒测试上的结果，我们还需要黑盒测试的结果。 

0.9288888888888889